In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
final_list=[]

# The last page's url to detect a termination condition
debugURL = 'https://dk.trustpilot.com/review/aura.dk?b=MjAxNi0wMS0wNVQxNjo1MTozMi4wMDBa'

# Let's go
url = 'https://dk.trustpilot.com'
nextPageAddress = "/review/aura.dk"

while True:
    
    print("Scanning for url-adresses: ", url+nextPageAddress)
    r=requests.get(url+nextPageAddress)
    soup = BeautifulSoup(r.text, 'lxml')
    for reviewCard in soup.find_all('div', class_='review-card'): # Find all section, class review content in soup
        
        
        #1 ----------------------------
        # Get the title  
        
        title = reviewCard.find('h2',class_='review-content__title') # Find title in paragraph
        if title is None:
            print("Variable title is None") # If there is no title, print it is none
        else:
            title = title.text.strip() 
            print("Title: ", title)
        
        #2 ----------------------------
        # Get the name         
        name = reviewCard.find('div', class_='consumer-information__name')
        print("name debug: ", name)
        if name is None:
            print("Variable name is None") # Checking if it exists
        else:
            name = name.text.strip() # From the whole HTML to only the text part
            print("Name: ", name)
            
        #3 ----------------------------
        # Get the author STATUS
        STATUS_HTML=reviewCard.find('div', class_='review-content-header__review-labels')
        if STATUS_HTML is None:
            print("Variable STATUS_HTML is None")
        else:
            STATUS_SCRIPT=STATUS_HTML.find('script', type='application/json')
            if STATUS_SCRIPT is None:
                print("No STATUS_SCRIPT is available")
                STATUS='UNKNOWN'
            else:
                STATUS_DATA=STATUS_SCRIPT.string
                #print("STATUS_DATA: ", STATUS_DATA)
                STATUS_JSON=json.loads(STATUS_DATA)
                print("STATUS_JSON: ", STATUS_JSON)
                if STATUS_JSON.get('isVerified', False):
                    print("User is verified")
                    STATUS = STATUS_JSON.get('verificationSource', 'SOURCE N/A')
                else:
                    print("User is not verified")
                    STATUS = STATUS_JSON.get('informationRequestStatus', 'STATUS N/A')
        print("STATUS: ", STATUS)
                
        #4 ----------------------------
        # Get the review content
        content = reviewCard.find('p', class_='review-content__text')
        if content is None: 
            print("Variable content is None") # Checking if it exists
        else:
            content = content.text.strip() # From the whole HTML to only the text part
            print("Content: ", content)
            
        #5 ----------------------------
        # Get the date and time     
        name = reviewCard.find('div', class_='consumer-information__name')
        print("name debug: ", name)
        if name is None:
            print("Variable name is None") # Checking if it exists
        else:
            name = name.text.strip() # From the whole HTML to only the text part
            print("Name: ", name)
        dateData = reviewCard.find('div',class_='review-content-header__dates')
        print("dateData: ", dateData)
        sC=dateData.find('script', type='application/json').string
        print("scriptContent: ", sC)
        jSD = json.loads(sC)
        print("JSON of this: ", jSD)
        
        date = "" 
        # print("publisheddate som index: ", jSD['publishedDate'])
        tempArray=jSD['publishedDate'].split('T')
        date=tempArray[0]
        time=tempArray[1]
        print("date: ", date)
        print("time: ", time)
        
        #6 ----------------------------
        # Get the star rating
        rating_content=reviewCard.find('div',class_='star-rating') #find div element, find class star rating
        print("rating_content:", rating_content)
        rating_img=rating_content.find('img')
        print("rating_img: ", rating_img)
        alt_text=rating_content.find('img')['alt']
        print("alt_text: ", alt_text)
        
        ratingArray=alt_text.split(' ')
        print("ratingArray: ", ratingArray)
        
        rating=ratingArray[0]  # Example ['2', 'stars:', 'Low', 'Medium']
        print("Rating: ", rating)

        final_list.append([title, content, name, STATUS, date, time, rating])
        
        
    # Finding the next page button 
    navigationHTML=soup.find('nav', class_='pagination-container') # soup is the whole site, then look for element 'nav'
    print("Navigational HTML: ", navigationHTML)
    
    nextPageLink=navigationHTML.find('a', class_='next-page') # Find 'a', and class "next page" in the HTML
    print("NextPageLink: ", nextPageLink)
    
    if nextPageLink is not None:
        nextPageAddress=nextPageLink['href'] # If I put class here, I would get a list of the classes 
        print("NextPageAddress: ", nextPageAddress)
    else:
        print("Did not find a nextPageAddress button")
        break # Exit while-loop
    df = pd.DataFrame(final_list, columns=['Title','Content','Name','Status','Date','Time','Rating'])
    
    # If going to debug, put a break here:
    #break
    
    